CODE FOR TRAINING CNN MODEL

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.api.utils import to_categorical
from keras.api.models import Sequential, load_model
from keras.api.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, Input
from keras.api.callbacks import LearningRateScheduler

# LOAD THE DATA
train = pd.read_csv("datasets\\train.csv")
test = pd.read_csv("datasets\\test.csv")

# PREPARE DATA FOR NEURAL NETWORK
Y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1)
X_train = X_train / 255.0
X_test = test / 255.0
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)
Y_train = to_categorical(Y_train, num_classes=10)

# PREVIEW IMAGES
plt.figure(figsize=(15, 4.5))
for i in range(30):
    plt.subplot(3, 10, i + 1)
    plt.imshow(X_train[i].reshape((28, 28)), cmap=plt.cm.binary)
    plt.axis('off')
plt.subplots_adjust(wspace=-0.1, hspace=-0.1)
plt.show()

# CREATE MORE IMAGES VIA DATA AUGMENTATION
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

# PREVIEW AUGMENTED IMAGES
X_train3 = X_train[9,].reshape((1, 28, 28, 1))
Y_train3 = Y_train[9,].reshape((1, 10))
plt.figure(figsize=(15, 4.5))
for i in range(30):
    plt.subplot(3, 10, i + 1)
    X_train2, Y_train2 = next(datagen.flow(X_train3, Y_train3))
    plt.imshow(X_train2[0].reshape((28, 28)), cmap=plt.cm.binary)
    plt.axis('off')
    if i == 9: X_train3 = X_train[11,].reshape((1, 28, 28, 1))
    if i == 19: X_train3 = X_train[18,].reshape((1, 28, 28, 1))
plt.subplots_adjust(wspace=-0.1, hspace=-0.1)
plt.show()

# Check if models are already saved and load them, or train if not found
nets = 5
model = [0] * nets
history = [0] * nets
models_directory = "trained_models"
os.makedirs(models_directory, exist_ok=True)

for j in range(nets):
    model_path = os.path.join(models_directory, f"model_{j}.keras") 
    if os.path.exists(model_path):
        print(f"Loading model_{j} from {models_directory}...")
        model[j] = load_model(model_path)
    else:
        print(f"Model {j} not found. Training the model...")
        model[j] = Sequential()
        model[j].add(Input(shape=(28, 28, 1)))
        model[j].add(Conv2D(32, kernel_size=3, activation='relu'))
        model[j].add(BatchNormalization())
        model[j].add(Conv2D(32, kernel_size=3, activation='relu'))
        model[j].add(BatchNormalization())
        model[j].add(Conv2D(32, kernel_size=5, strides=2, padding='same', activation='relu'))
        model[j].add(BatchNormalization())
        model[j].add(Dropout(0.4))

        model[j].add(Conv2D(64, kernel_size=3, activation='relu'))
        model[j].add(BatchNormalization())
        model[j].add(Conv2D(64, kernel_size=3, activation='relu'))
        model[j].add(BatchNormalization())
        model[j].add(Conv2D(64, kernel_size=5, strides=2, padding='same', activation='relu'))
        model[j].add(BatchNormalization())
        model[j].add(Dropout(0.4))

        model[j].add(Conv2D(128, kernel_size=4, activation='relu'))
        model[j].add(BatchNormalization())
        model[j].add(Flatten())
        model[j].add(Dropout(0.4))
        model[j].add(Dense(10, activation='softmax'))

        # Compile and train the model
        model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

        X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size=0.1)

        epo = 30
        history[j] = model[j].fit(
            datagen.flow(X_train2, Y_train2, batch_size=64),
            epochs=epo,
            steps_per_epoch=len(X_train2) // (epo*64),  # Ensure the number of steps is appropriate
            validation_data=(X_val2, Y_val2),
            callbacks=[LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)],
            verbose=0
        )

        print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j + 1, epo, max(history[j].history['accuracy']), max(history[j].history['val_accuracy'])))
        20012002
        # Save the trained model
        model[j].save(model_path)
        print(f"Model {j} saved as {model_path}.")

# ENSEMBLE PREDICTIONS
results = np.zeros((X_test.shape[0], 10))
for j in range(nets):
    results = results + model[j].predict(X_test)
results = np.argmax(results, axis=1)
results = pd.Series(results, name="Label")

# PREVIEW PREDICTIONS
plt.figure(figsize=(15, 6))
for i in range(30):
    plt.subplot(4, 10, i + 1)
    plt.imshow(X_test[i].reshape((28, 28)), cmap=plt.cm.binary)
    plt.title("predict=%d" % results[i], y=0.9)
    plt.axis('off')
plt.subplots_adjust(wspace=0.3, hspace=-0.1)
plt.show()

CODE FOR TESTING CNN MODELS

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def preprocess_image(image_path):
    """
    Load and preprocess the image for prediction.
    - Convert the image to grayscale.
    - Resize it to 28x28 pixels.
    - Invert pixel values if necessary.
    """
    # Load the image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image at {image_path} not found or unable to load.")
    
    # Resize to 28x28
    img = cv2.resize(img, (28, 28))
    
    # Invert the pixel values
    img = 255 - img 

    # Normalize pixel values to [0, 1].
    img = img / 255.0

    # Reshape to (1, 28, 28, 1) for the model input
    img = img.reshape(1, 28, 28, 1)
    
    return img

def save_pixel_data(image, image_name, output_folder):
    """
    Save the pixel values of the image in a CSV file.
    - The data will be column-wise with column names like 'pixel0', 'pixel1', ..., 'pixel783'.
    """
    # Flatten the image to 1D array (784 pixels)
    pixel_data = image.flatten()
    
    # Create a DataFrame with pixel data
    pixel_columns = [f"pixel{i}" for i in range(784)]
    pixel_df = pd.DataFrame([pixel_data], columns=pixel_columns)
    
    # Save the DataFrame to a CSV file
    csv_filename = os.path.join(output_folder, f"{image_name}_pixels.csv")
    pixel_df.to_csv(csv_filename, index=False)

def predict_and_save_pixels(models, folder_path, image_count):
    """
    Predict results, visualize each image, and save the pixel data in CSV files.
    - `models`: List of trained models.
    - `folder_path`: Folder containing images.
    - `image_count`: Number of images to process.
    """
    # Create a folder to store generated CSV files if it doesn't exist
    output_folder = "generated_csv_files"
    os.makedirs(output_folder, exist_ok=True)

    for i in range(image_count):
        image_name = f"i{i}.png"
        image_path = os.path.join(folder_path, image_name)
        
        try:
            # Preprocess the image
            img = preprocess_image(image_path)
            
            # Initialize results array
            results = np.zeros((1, 10))
            
            # Use each model in the ensemble to predict
            for model in models:
                # Ensure the model is compiled if necessary
                if not model.compiled:
                    model.compile()
                
                results += model.predict(img)

            
            # Normalize probabilities
            probabilities = results / len(models)
            probabilities = probabilities.flatten()
            
            # Get the predicted label
            predicted_label = np.argmax(probabilities)
            
            # Display the image with its probabilities
            plt.figure(figsize=(15, 6))
            plt.subplot(4, 10, 1)
            plt.imshow(img.reshape(28, 28), cmap=plt.cm.binary)
            plt.axis('off')
            prob_str = "\n".join([f"Probability of {i}: {p:.10f}" for i, p in enumerate(probabilities)])
            plt.title(f"{image_name}: Predicted: {predicted_label}\nProbabilities:\n{prob_str}")
            plt.show()
            
            # Save the pixel data to a CSV file
            save_pixel_data(img, image_name, output_folder)
        
        except Exception as e:
            print(f"Error processing {image_name}: {e}")

folder_path = "Predict Image"  # Folder containing images i0.png to i25.png
image_count = 25  # Total number of images to process (i0 to i25)

try:
    predict_and_save_pixels(model, folder_path, image_count)
except Exception as e:
    print(f"Error: {e}")